In [95]:
import requests
from bs4 import BeautifulSoup
import csv
import re
#지금 문제: llii 리스트 저장 append로 고치기

In [79]:
def make_breaktime(name,day,opentime,closed,breaktime):
    #쉬는시간#########################################
    breakstart = ''
    breakend = ''
    breaksplit = breaktime.split(' ')
    for a in range(len(breaksplit)):
        if (breaksplit[0][0]==0 or breaksplit[0][0]==1 or breaksplit[0][0]==2):
            breakstart = breaksplit[0]
            breakend = breaksplit[2]
        elif (breaksplit[a][0]=='월'or breaksplit[a][0]=='화'or breaksplit[a][0]=='수'or breaksplit[a][0]=='목'or breaksplit[a][0]=='금'or breaksplit[a][0]=='토'or breaksplit[a][0]=='일'):
            days = hangul.findall(breaksplit[a])
            if (len(days)==1):
                if (day==days[0]): #현재 요일일 경우에만 저장함
                    breakstart = breaksplit[a+1]
                    breakend = breaksplit[a+3]
                else:
                    continue
            elif (len(days)==2):
                start = daylist.index(days[0])
                daylist_trans = daylist[start:]
                end = daylist_trans.index(days[1])+start
                for m in range(start-end+1):
                    if (day==daylist[start+m]): #현재 요일일 경우에만 저장함
                        breakstart = breaksplit[a+1]
                        breakend = breaksplit[a+3]
                    else:
                        continue
            else: #월/화/수/목 이런 경우
                for m in range(len(days)):
                    if (day==days[m]): #현재 요일일 경우에만 저장함
                        breakstart = breaksplit[a+1]
                        breakend = breaksplit[a+3]
                    else:
                        continue
    print([name,day,opentime,closed,breakstart,breakend])
    return [[name,day,opentime,closed,breakstart,breakend]]
    ######################################################

In [87]:
def daybyday(days,salesplit,breaktime,llii,k):
    start=-1
    end=-1
    #ex)일:
    if (len(days)==1):
        day = days[0]
        opentime = salesplit[k+1]
        closed = salesplit[k+3]
        if not (breaktime==''):
            print('this is breaktime: '+make_breaktime(name,day,opentime,closed,breaktime))
            llii += make_breaktime(name,day,opentime,closed,breaktime)
        print(llii)
    #ex)금-토:
    elif (len(days)==2):
        start = daylist.index(days[0])
        daylist_trans = daylist[start:]
        end = daylist_trans.index(days[1])+start
        for m in range(end-start+1):
            day = daylist[start+m]
            opentime = salesplit[k+1]
            closed = salesplit[k+3]
            if not (breaktime==''):
                print('this is breaktime: '+make_breaktime(name,day,opentime,closed,breaktime))
                llii += make_breaktime(name,day,opentime,closed,breaktime)
            print(llii)
    #월/화/수/목:
    else:
        for m in range(len(days)):
            day = days[m]
            opentime = salesplit[k+1]
            closed = salesplit[k+3]
            print('3opentime:'+opentime,'closed: '+closed)
            if not (breaktime==''):
                print('this is breaktime: '+make_breaktime(name,day,opentime,closed,breaktime))
                llii += make_breaktime(name,day,opentime,closed,breaktime)
            print(llii)
    return start,end

In [94]:
number=1
lii=[] #상세주소url을담을리스트
url="https://www.mangoplate.com/search/"
headers = {'User-Agent' : 'Mozilla/5.0'}
response = requests.get(url, headers=headers)

soup = BeautifulSoup(response.text, 'html.parser')
    
keyword = '가로수길' #코드 다 돌아가면 키워드 바꿔서 새로 실행
    
for p in range (1): #########################################10임!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

    url = f"https://www.mangoplate.com/search/{keyword}?keyword={keyword}&page={number}"

    headers = {'User-Agent' : 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)

    soup = BeautifulSoup(response.content.decode('utf-8','replace'), 'html.parser')
    data = soup.select("li.server_render_search_result_item > div.list-restaurant-item")

    
        #기본정보(가게이름/평점/대략적인위치/카테고리/리뷰개수)
    for item in data:
            
            #기본페이지에서 상세페이지url가져오기
        urlllll = (str(item.select_one('a.only-desktop_not')).split(' ')[2][5:])[1:-1]

        urll = str("https://www.mangoplate.com"+urlllll)
        lii+=[urll] #url만 있는 리스트를 만들었음. 상세정보 url은 이 리스트에서 가져올거임
        
    y = 0
    llii = []
    daylist=['월','화','수','목','금','토','일','월','화','수','목','금','토'] #'토-일'로 쓰기도 하고 '일-월'로 쓰기도 해서 원형리스트처럼 만들어봄
    for u in range(len(lii)):
        url = lii[y]

        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        data1 = soup.select("section.restaurant-detail")
        data2 = soup.select("table.info")
            
        holidays = ''
        breaktime = ''
        sales = ''

        for d1 in data1:
            name = d1.select_one("h1.restaurant_name").text

        for items in data2:
            trlist = items.select("tr")
            for i,tr in enumerate(trlist):
                if (trlist[i].select_one("th").text=='영업시간'):
                    timeinfo = trlist[i]
                    sales = timeinfo.text
                    sales = sales.replace('영업시간','')
                    sales = sales.replace('\n','')
                elif (trlist[i].select_one("th").text=='쉬는시간'):
                    timeinfo = trlist[i]
                    breaktime = timeinfo.text
                    breaktime = breaktime.replace('쉬는시간','')
                    breaktime = breaktime.replace('\n','')
                elif (trlist[i].select_one("th").text=='휴일'):
                    timeinfo = trlist[i]
                    holidays = timeinfo.text
                    holidays = holidays.replace('휴일','')
                    holidays = holidays.replace('\n','')
                else:
                    continue
        print(sales+'그리고'+breaktime+'그리고'+holidays)
        # 휴무일 레코드
        if (holidays!=''):
            holist = holidays.split(',')
            for j in range(len(holist)):
                llii += [[name,holist[j],'','','','']]
            
        hangul = re.compile('[ㄱ-ㅣ가-힣]+') # 띄어쓰기를 포함한 모든 한글

        #영업일 레코드
        if not (sales==''):
            salesplit = sales.split(' ')
            for k in range(len(salesplit)):                
                #영업일 맨 첫글자가 요일 #ex)일: 11:00 - 22:00 금-토: 11:00 - 23:00
                if (salesplit[k][0]=='월' or salesplit[k][0]=='화' or salesplit[k][0]=='수' or salesplit[k][0]=='목' or salesplit[k][0]=='금' or salesplit[k][0]=='토' or salesplit[k][0]=='일'):
                    days = hangul.findall(salesplit[k])
                    daybyday(days,salesplit,breaktime,llii,k)
                else: #영업일 맨 첫글자가 요일이 아님
                    #ex)유동적
                    if (salesplit[0]=='유동적'):
                        day = '유동적'
                        print([name,day,day,day,'',''])
                        llii += [[name,day,day,day,'','']]
                        
                    boo = False
                    for w in range(7):
                        try: #영업일 안에 요일이 있음 #ex)11:30 - 01:00 토-일: 14:00 - 01:00
                            ind = list(sales).index(daylist[w])
                            days = hangul.findall(salesplit[k])
                            start,end = daybyday(days,salesplit,breaktime,llii,k)
                            daily = daylist
                            for m in range(end-start+1):
                                daily.remove(daylist[start+m])
                            for b in range(len(daily)):
                                day = daily[b]
                                opentime = salesplit[k+1]
                                closed = salesplit[k+3]
                                print('3opentime:'+opentime,'closed: '+closed)
                                if not (breaktime==''):
                                    print('this is breaktime: '+make_breaktime(name,day,opentime,closed,breaktime))
                                    llii += make_breaktime(name,day,opentime,closed,breaktime)
                                print(llii)
                            boo = True
                            break
                        except ValueError:
                            continue
                        
                    if not (boo): #영업일 안에 요일이 없음
                        #ex)11:30 - 21:30
                        if (salesplit[0][0]==str(0) or salesplit[0][0]==str(1) or salesplit[0][0]==str(2)):
                            for n in range(7):
                                day = daylist[n]
                                opentime = salesplit[0]
                                closed = salesplit[2]
                                print('0opentime:'+opentime,'closed: '+closed)
                                if not (breaktime==''):
                                    print('this is breaktime: '+make_breaktime(name,day,opentime,closed,breaktime))
                                    llii += make_breaktime(name,day,opentime,closed,breaktime)
                                print(llii)
            
        y += 1
        
    number += 1


# with open('mango_res_1.csv', 'a', newline='') as f:
#     write = csv.writer(f)
#     write.writerows(llii)
    

print("complete ♡ 오츠카레사마데시따 ~☆")

18:00 - 01:00그리고그리고일
0opentime:18:00 closed: 01:00
[['미라이', '일', '', '', '', '']]
0opentime:18:00 closed: 01:00
[['미라이', '일', '', '', '', '']]
0opentime:18:00 closed: 01:00
[['미라이', '일', '', '', '', '']]
0opentime:18:00 closed: 01:00
[['미라이', '일', '', '', '', '']]
0opentime:18:00 closed: 01:00
[['미라이', '일', '', '', '', '']]
0opentime:18:00 closed: 01:00
[['미라이', '일', '', '', '', '']]
0opentime:18:00 closed: 01:00
[['미라이', '일', '', '', '', '']]
0opentime:18:00 closed: 01:00
[['미라이', '일', '', '', '', '']]
0opentime:18:00 closed: 01:00
[['미라이', '일', '', '', '', '']]
0opentime:18:00 closed: 01:00
[['미라이', '일', '', '', '', '']]
0opentime:18:00 closed: 01:00
[['미라이', '일', '', '', '', '']]
0opentime:18:00 closed: 01:00
[['미라이', '일', '', '', '', '']]
0opentime:18:00 closed: 01:00
[['미라이', '일', '', '', '', '']]
0opentime:18:00 closed: 01:00
[['미라이', '일', '', '', '', '']]
0opentime:18:00 closed: 01:00
[['미라이', '일', '', '', '', '']]
0opentime:18:00 closed: 01:00
[['미라이', '일', '', '', '', '']]
0op

TypeError: can only concatenate str (not "list") to str

In [4]:
number=1
lii=[] #상세주소url을담을리스트
url="https://www.mangoplate.com/search/"
headers = {'User-Agent' : 'Mozilla/5.0'}
response = requests.get(url, headers=headers)

soup = BeautifulSoup(response.text, 'html.parser')
    
keyword = '가로수길' #코드 다 돌아가면 키워드 바꿔서 새로 실행

url = f"https://www.mangoplate.com/search/{keyword}?keyword={keyword}&page={number}"

headers = {'User-Agent' : 'Mozilla/5.0'}
response = requests.get(url, headers=headers)

soup = BeautifulSoup(response.content.decode('utf-8','replace'), 'html.parser')
data = soup.select("li.server_render_search_result_item > div.list-restaurant-item")


#기본정보(가게이름/평점/대략적인위치/카테고리/리뷰개수)
for item in data:
            
    #기본페이지에서 상세페이지url가져오기
    urlllll = (str(item.select_one('a.only-desktop_not')).split(' ')[2][5:])[1:-1]

    urll = str("https://www.mangoplate.com"+urlllll)
    lii+=[urll] #url만 있는 리스트를 만들었음. 상세정보 url은 이 리스트에서 가져올거임

    y = 0
    llii = []
    daylist=['월','화','수','목','금','토','일','월','화','수','목','금','토'] #'토-일'로 쓰기도 하고 '일-월'로 쓰기도 해서 원형리스트처럼 만들어봄
    for i in range(len(lii)):
        url = lii[y]

        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        data1 = soup.select("section.restaurant-detail")
        data2 = soup.select("table.info")
            
        holidays = ''
        breaktime = ''
        sales = ''